In [1]:
using StatsModels,DataFrames,GMMInference, Distributions, JuMP, Ipopt, LinearAlgebra,NLPModels, Distributions, Optim, ForwardDiff, ECOS 

┌ Info: Precompiling StatsModels [3eaba693-59b7-5ba5-a881-562e759f1c8d]
└ @ Base loading.jl:1278
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling GMMInference [bac0df68-cab7-482a-aa4b-d3925df9b168]
└ @ Base loading.jl:1278
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1278


# HW 7, Jan Rosa
Here I write a simple interface for gmm model IVLogitShare, using the interface from StatsModels. Unfortunately, the interface for now works only on dataframes (StatModels operate only on this type) and with for one of the implemented models IVLogitShare

In [7]:

mutable struct ivModelFrame
    #structure for the iv, ModelFrame = class from StatsModels
    first_stage::ModelFrame
    second_stage::ModelFrame
end

function ivModelFrame(f1::FormulaTerm, f2::FormulaTerm,data::DataFrame)
    #declaration of ivModelFrame using two formulas for each stage and one dataframe 
    first_stage = ModelFrame(f1, data)
    second_stage = ModelFrame(f2, data)
    return(ivModelFrame(first_stage,second_stage ))
end 


#These function was designed to be something similar to lm in GLS
function gmm(formula1, formula2,data; model = "ivlogit") #I only wrote implementation for IVLogitShare "ivlogit" here
    
    model_frame = ivModelFrame(formula1, formula2,data) 
    first_stage = model_frame.first_stage
    
    X = ModelMatrix(model_frame.first_stage) #Here I use function form StatModels which was very useful with building matrix of variables of rhs of regression equation, 
                                                #with constants, interactions etc 
    Z = ModelMatrix(model_frame.second_stage)
    
    x = convert(Matrix{Float64}, X.m) #convert matrix of data to the type used in GMM 
    z = convert(Matrix{Float64}, Z.m)
    
    dict = first_stage.schema #take y using dictionary 
    Y = string(get(dict,formula1.lhs,  nothing)) 
    y_indf = df[ ( names(df) .== Y ) ] 
    
    y = convert(Matrix{Float64}, y_indf)  #convert y to the datatype used in GMM 
    y = vec(y)
    
    #define model
    if(model == "ivlogit")
        gmm_model = IVLogitShare(x,y,z)
    else
        gmm_model= nothing
    end
    
    #Find coefficients as in the lm function in GLS
    
    n = number_observations(gmm_model)
    optim_model = gel_jump_problem(gmm_model)
    
    set_optimizer(optim_model, with_optimizer(Ipopt.Optimizer, print_level=5))
    set_start_value.(optim_model[:θ], 0.0)
    set_start_value.(optim_model[:p], 1/n)
    optimize!(optim_model)
    
    @show value.(optim_model[:θ])
    @show value.(optim_model[:p][1:10])
    
    return(θ = value.(optim_model[:θ]) )
end

gmm (generic function with 1 method)

In [8]:
#Qucik and random test to see if interface works

df = DataFrame(y = rand(100), a = rand(100), b = randn(100), c = randn(100), d = randn(100))
Istage = @formula(y ~a+c)
IIstage = @formula(c ~a+b+d)

Model_with_interface = gmm(Istage,IIstage,df)



This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2900
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1300

Total number of variables............................:      103
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      100
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

3-element Array{Float64,1}:
  0.11790491142755562
 -0.25627285419630635
 -0.7048990034040284